#  Cálculo de Griegas para Opciones Europeas
usando AAD con Tensorflow2

En este notebook se busca explorar el cálculo de algunas sensibilidades de primer orden de las opciones europeas, haciendo uso de Adjoint Algorithmic Differentiation, por medio de Tensorflow 2.

Para este fin debemos recordar el modelo de valoración Black-Scholes-Merton para opciones europeas como:

$V = \psi S \Phi(\psi d1) - \psi K e^{-r T} \Phi(\psi d2)$

con:

* $S$ : Precio del subyacente
* $K$ : Strike del contrato
* $T$ : Plazo de maduración del contrato
* $r$ : Tasa libre de riesgo
* $\psi$ : variable para determinar si es opción Call (1) o Put (-1)
* $\sigma$ : volatilidad implícita

* $d1 = \frac{log(\frac{S}{K}) + (r + \frac{\sigma^2}{2})T}{\sigma \sqrt{T}}$

* $d2 = d1 - \sigma \sqrt{T}$

Con lo anterior en mente, vale la pena recordar la definición de las sensibilidades de primer orden:

* $\Delta = \frac{\partial{V}}{\partial{S}} = \Phi(d_+)$
* $\nu = \frac{\partial{V}}{\partial{\sigma}} = \frac{e^{-r T} S \sqrt{T}}{\sqrt{2\pi}} e^{\frac{-d_1^2}{2}}$
* $\Theta = \frac{\partial{V}}{\partial{T}} = \frac{1}{T}\left( \frac{S \sigma}{2 \sqrt{2\pi T}} e^{-\frac{d_1^2}{2}} - \psi r K e^{-r T}\Phi(\psi d_2)\right)$
* $\rho = \frac{\partial{V}}{\partial{r}} = \psi K T e^{-r T} \Phi(\psi d_2) $

Para lo anterior vale la pena recordar que:

$\Phi(x) = \displaystyle\int_{-\infty}^{x} \frac{1}{\sqrt{2\pi}\sigma} e^{\frac{-(t - \mu)^2}{2\sigma^2}} \,dx$

por lo tanto para nuestro $\Phi^{\prime}(x)$, con $x \sim N(0,1)$, será:

$\Phi^{\prime}(x) = \frac{1}{\sqrt{2\pi}}e^{\frac{-x^2}{2}}$

## librerías

Se usan las librerías de tensorflow y tensorflow_probability, cuyo objetivo es construir el grafo (DAG, Directed Acyclic Graph) donde se aplicará la diferenciación algorítmica backward o también conocida como AAD (Adjoint Algorithmic Differentiation).

In [148]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

## Modelo Black-Scholes-Merton
Función en el contexto de Tensorflow con la especificación del modelo de BSM

In [2]:
@tf.function(experimental_relax_shapes=True)
def black_scholes_merton(S, K, T, sigma, rf, psi):
    """
    Black-Scholes-Merton model Tensorflow

    Parameters
    ------------
    S       : Underliying asset Price [tensorflow.Variable]
    K       : Strike Price [tensorflow.Variable]
    T       : Time to maturity [tensorflow.Variable]
    sigma   : implied volatility [tensorflow.Variable]
    rf      : risk free rate [tensorflow.Variable]
    psi     : Call (1) - Put (-1) [tensorflow.Variable]
    """
    phi = tfp.distributions.Normal(0,1).cdf
    sigma_sqrt_T = sigma * tf.math.sqrt(T)

    d1 = (tf.math.log(S/K) + (rf + 0.5*sigma**2)*T) / sigma_sqrt_T
    d2 = d1 - sigma_sqrt_T
    option_price = psi * S * phi(psi*d1) - psi * K * tf.exp(-rf*T) * phi(psi*d2)
    return option_price

## Función Wrapper
Esta función será una función envoltorio que permitirá el \<cast\> de valores de entrada a variables en el contexto de Tensorflow.

In [3]:
def calculate_BSM(S, K, T, sigma, rf, psi):
    """
    Function which price a european option and their greeks
    
    Parameters
    ------------
    S       : Underliying asset Price
    K       : Strike Price
    T       : Time to maturity
    sigma   : implied volatility
    rf      : risk free rate
    psi     : Call (1) - Put (-1)
    """
    if not tf.is_tensor(S):
        S = tf.Variable(S)
    if not tf.is_tensor(K):
        K = tf.Variable(K)
    if not tf.is_tensor(T):
        T = tf.Variable(T)
    if not tf.is_tensor(sigma):
        sigma = tf.Variable(sigma)
    if not tf.is_tensor(rf):
        rf = tf.Variable(rf)
    if not tf.is_tensor(psi):
        psi = tf.Variable(psi)

    variables = {'S':S, 'K':K, 'T':T, 'sigma':sigma, 'rf':rf, 'psi':psi}
    greeks = {'S':'delta', 'K':'d_K', 'T':'tetha', 'sigma':'vega', 'rf':'rho', 'psi':'d_psi'}

    with tf.GradientTape() as g1:
        price = black_scholes_merton(**variables)
    d_price = g1.gradient(price, variables)
    d_price = {greeks[k]: v.numpy() for k,v in d_price.items()}
    return dict(price=price.numpy(), d_price=d_price)

## Implementación
La implementación permite enviar bien sea la información facial de solamente una opción o varias, en forma de listas, según se necesite.

Ejemplo 1:
* $S$ : 10
* $K$ : 11
* $T$ : 1 año
* $r$ : 1%
* $\psi$ : 1 (Call)
* $\sigma$ : 3%

In [4]:
resultado_ej_1 = calculate_BSM(S=[10.], K=[11.,], T=[1.], sigma=[0.03], rf=[0.01], psi=[1.])

In [5]:
print(f"Precio\t : {resultado_ej_1['price'][0]}")
print(f"Delta\t : {resultado_ej_1['d_price']['delta'][0]}")
print(f"Vega\t : {resultado_ej_1['d_price']['vega'][0]}")
print(f"Rho\t : {resultado_ej_1['d_price']['rho'][0]}")
print(f"Theta\t : {-resultado_ej_1['d_price']['tetha'][0]}")

Precio	 : 0.00020557455718517303
Delta	 : 0.0023369055707007647
Vega	 : 0.07301333546638489
Rho	 : 0.023163482546806335
Theta	 : -0.0013268347829580307


Ejemplo 2 (más de una opción a la vez):

Opción 1
* $S_1$ : 10
* $K_1$ : 10
* $T_1$ : 1 año
* $r_1$ : 1%
* $\psi_1$ : 1 (Call)
* $\sigma_1$ : 3%

Opción 2
* $S_2$ : 10
* $K_2$ : 12
* $T_2$ : 1.5 años
* $r_2$ : 1%
* $\psi_2$ : -1 (Put)
* $\sigma_2$ : 3.2%

Opción 3
* $S_3$ : 10
* $K_3$ : 9.5
* $T_3$ : 0.4 años
* $r_3$ : 1%
* $\psi_3$ : -1 (Put)
* $\sigma_3$ : 2.7%

In [8]:
resultado_ej_2 = calculate_BSM(
                        S=[10.,10.,10.,10.], # mercado en t0, igual para todas
                        K=[11.,8.9,12.,9.5],
                        T=[1.2,0.7,1.5,0.4],
                        sigma=[0.03,0.031,0.032,0.027],
                        rf=[0.01,0.01,0.01,0.01], # mercado en t0, igual para todas
                        psi=[1.,1.,-1.,-1.])

In [9]:
for option_i in range(len(resultado_ej_2['price'])):
    print('-- -- -- -- -- -- -- -- -- -- -- -- -- ')
    print(f'Opción {(option_i+1)}')
    print('-- -- -- -- -- -- -- -- -- -- -- -- -- ')
    print(f"Precio\t : {resultado_ej_2['price'][option_i]}")
    print(f"Delta\t : {resultado_ej_2['d_price']['delta'][option_i]}")
    print(f"Vega\t : {resultado_ej_2['d_price']['vega'][option_i]}")
    print(f"Rho\t : {resultado_ej_2['d_price']['rho'][option_i]}")
    print(f"Theta\t : {-resultado_ej_2['d_price']['tetha'][option_i]}")
    print('')

-- -- -- -- -- -- -- -- -- -- -- -- -- 
Opción 1
-- -- -- -- -- -- -- -- -- -- -- -- -- 
Precio	 : 0.0006155781447887421
Delta	 : 0.005890722386538982
Vega	 : 0.18324600160121918
Rho	 : 0.06994997709989548
Theta	 : -0.002873491495847702

-- -- -- -- -- -- -- -- -- -- -- -- -- 
Opción 2
-- -- -- -- -- -- -- -- -- -- -- -- -- 
Precio	 : 1.1620826721191406
Delta	 : 0.9999991059303284
Vega	 : 3.719647793332115e-05
Rho	 : 6.186535835266113
Theta	 : -0.08837991207838058

-- -- -- -- -- -- -- -- -- -- -- -- -- 
Opción 3
-- -- -- -- -- -- -- -- -- -- -- -- -- 
Precio	 : 1.8213434219360352
Delta	 : -0.9999892711639404
Vega	 : 0.0005851517780683935
Rho	 : -17.731857299804688
Theta	 : 0.11820612847805023

-- -- -- -- -- -- -- -- -- -- -- -- -- 
Opción 4
-- -- -- -- -- -- -- -- -- -- -- -- -- 
Precio	 : 2.670520916581154e-05
Delta	 : -0.0005840239464305341
Vega	 : 0.012976610101759434
Rho	 : -0.0023467775899916887
Theta	 : -0.0003792911593336612



Monte Carlo comparison, ... coming soon.

<hr>

This could not to be the final version, so if you have some recommendation or comment about this, I will be grateful to hear it, using e-mail <b>craquinterogo@unal.edu.co</b> or <b>cristian.quintero@est.uexternado.edu.co</b>